# Lab 06 — Classification

> **강의 시간:** 약 1시간  
> **주제:** 로지스틱 회귀부터 PyTorch 소프트맥스 분류기까지

---

## 학습 목표

| # | 목표 |  
|---|---| 
| 1 | Sigmoid / 결정 경계 이해 및 시각화 |  
| 2 | 소프트맥스와 크로스 엔트로피 손실 |  
| 3 | Iris 데이터셋으로 PyTorch 분류기 구현 |  
| 4 | 평가 지표: 정확도, 정밀도, 재현율, F1, 혼동 행렬 |  
| 5 | Exercise: MNIST 소프트맥스 분류기 |  

---

**데이터셋:** Iris (sklearn) — 붓꽃 3종 분류 / MNIST — 손글씨 숫자 분류

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris, fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve
)

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

rng = np.random.default_rng(42)
torch.manual_seed(42)
print('PyTorch:', torch.__version__)

---
## Part 1. 로지스틱 회귀와 결정 경계

### 1-1. 왜 선형 회귀로 분류를 못 할까?

**분류(Classification)** 는 출력이 **이산적인 클래스 레이블**입니다.  
선형 회귀($\hat{y} = \mathbf{w}^\top\mathbf{x}$)는 $(-\infty, +\infty)$ 범위의 값을 내놓기 때문에  
확률(0~1)을 나타내기에 부적합합니다.

**로지스틱 회귀**는 선형 출력을 **시그모이드 함수**로 압축합니다:

$$\hat{p} = \sigma(z) = \frac{1}{1 + e^{-z}}, \quad z = \mathbf{w}^\top\mathbf{x} + b$$

- $\hat{p} \geq 0.5$ → 클래스 1로 예측
- $\hat{p} < 0.5$ → 클래스 0으로 예측

In [ ]:
# 시그모이드 함수 시각화
z = np.linspace(-6, 6, 300)
sigmoid = 1 / (1 + np.exp(-z))
grad    = sigmoid * (1 - sigmoid)   # 도함수

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(z, sigmoid, color='steelblue', lw=2.5)
axes[0].axhline(0.5, color='tomato', linestyle='--', lw=1.5, label='임계값 0.5')
axes[0].axvline(0.0, color='gray', linestyle='--', lw=1.0)
axes[0].fill_between(z, 0.5, sigmoid, where=(sigmoid >= 0.5),
                     alpha=0.15, color='steelblue', label='클래스 1 영역')
axes[0].fill_between(z, sigmoid, 0.5, where=(sigmoid < 0.5),
                     alpha=0.15, color='tomato', label='클래스 0 영역')
axes[0].set_title('시그모이드 함수 $\\sigma(z)$')
axes[0].set_xlabel('z (선형 출력)')
axes[0].set_ylabel('$\\hat{p}$ (클래스 1 확률)')
axes[0].legend()
axes[0].set_ylim(-0.05, 1.05)

axes[1].plot(z, grad, color='seagreen', lw=2.5, label="$\\sigma'(z) = \\sigma(1-\\sigma)$")
axes[1].axvline(0, color='gray', linestyle='--', lw=1.0)
axes[1].set_title('시그모이드 도함수')
axes[1].set_xlabel('z')
axes[1].set_ylabel('그래디언트')
axes[1].legend()

plt.tight_layout()
plt.show()

print('σ(0) =', 1/(1+np.exp(0)))
print('σ(-∞) → 0,  σ(+∞) → 1')
print('z=0일 때 그래디언트 최대:', 0.25)

### 1-2. 이진 분류 — Iris 데이터 (Setosa vs Non-Setosa)

Iris 데이터셋에서 **꽃잎 길이(petal length), 꽃잎 너비(petal width)** 2개 특성으로  
**Setosa(0) vs Non-Setosa(1)** 를 분류합니다.

**결정 경계(Decision Boundary):** $\mathbf{w}^\top\mathbf{x} + b = 0$ 인 선  
→ 이 선을 기준으로 두 클래스가 나뉩니다.

In [ ]:
# Iris 데이터 로드 및 이진 분류 준비
iris   = load_iris()
X_iris = iris.data[:, 2:4].astype(np.float32)   # petal length, petal width
y_bin  = (iris.target != 0).astype(np.float32)   # Setosa=0, 나머지=1

scaler  = StandardScaler()
X_s     = scaler.fit_transform(X_iris)

X_tr, X_te, y_tr, y_te = train_test_split(
    X_s, y_bin, test_size=0.2, random_state=42, stratify=y_bin
)

print(f'전체 샘플: {len(X_iris)}')
print(f'클래스 분포 — Setosa: {(y_bin==0).sum()}, Non-Setosa: {(y_bin==1).sum()}')

# 산점도
fig, ax = plt.subplots(figsize=(7, 5))
colors_cls = ['steelblue', 'tomato']
labels_cls = ['Setosa (0)', 'Non-Setosa (1)']
for c, color, label in zip([0, 1], colors_cls, labels_cls):
    mask = y_bin == c
    ax.scatter(X_iris[mask, 0], X_iris[mask, 1],
               color=color, label=label, s=60, edgecolors='k', alpha=0.8)
ax.set_xlabel('꽃잎 길이 (cm)')
ax.set_ylabel('꽃잎 너비 (cm)')
ax.set_title('Iris — Setosa vs Non-Setosa')
ax.legend()
plt.tight_layout()
plt.show()

### 1-3. 이진 크로스 엔트로피 손실 (BCE Loss)

MSE 대신 **Binary Cross-Entropy**를 사용합니다:

$$\mathcal{L}_{\text{BCE}} = -\frac{1}{n}\sum_{i=1}^{n}\left[ y_i \log \hat{p}_i + (1-y_i)\log(1-\hat{p}_i) \right]$$

- $y=1$이면 $-\log\hat{p}$: 예측 확률이 높을수록 손실↓
- $y=0$이면 $-\log(1-\hat{p})$: 예측 확률이 낮을수록 손실↓

In [ ]:
# BCE Loss 시각화
p = np.linspace(1e-6, 1 - 1e-6, 300)
bce_y1 = -np.log(p)        # y=1일 때
bce_y0 = -np.log(1 - p)    # y=0일 때

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(p, bce_y1, color='steelblue', lw=2.5, label='y=1: $-\\log(\\hat{p})$')
ax.plot(p, bce_y0, color='tomato',    lw=2.5, label='y=0: $-\\log(1-\\hat{p})$')
ax.axvline(0.5, color='gray', linestyle='--', lw=1.2)
ax.set_xlim(0, 1)
ax.set_ylim(0, 5)
ax.set_xlabel('예측 확률 $\\hat{p}$')
ax.set_ylabel('손실')
ax.set_title('Binary Cross-Entropy Loss')
ax.legend()
plt.tight_layout()
plt.show()

# PyTorch BCE 예시
y_true_ex = torch.tensor([1.0, 0.0, 1.0, 0.0])
y_pred_ex = torch.tensor([0.9, 0.1, 0.4, 0.8])
bce = nn.BCELoss()(y_pred_ex, y_true_ex)
print(f'BCE Loss 예시: {bce.item():.4f}')
print(f'  예측 맞음 (0.9→1, 0.1→0): 손실 작음')
print(f'  예측 틀림 (0.4→1, 0.8→0): 손실 큼')

In [ ]:
# PyTorch로 로지스틱 회귀 학습
X_tr_t = torch.tensor(X_tr)
y_tr_t = torch.tensor(y_tr)
X_te_t = torch.tensor(X_te)
y_te_t = torch.tensor(y_te)

log_reg = nn.Sequential(
    nn.Linear(2, 1),
    nn.Sigmoid()
)
optimizer = torch.optim.Adam(log_reg.parameters(), lr=0.05)
criterion = nn.BCELoss()

losses = []
for epoch in range(200):
    log_reg.train()
    optimizer.zero_grad()
    pred  = log_reg(X_tr_t).squeeze()
    loss  = criterion(pred, y_tr_t)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

# 결정 경계 시각화
log_reg.eval()
with torch.no_grad():
    # 그리드 생성
    x0_min, x0_max = X_s[:, 0].min()-0.5, X_s[:, 0].max()+0.5
    x1_min, x1_max = X_s[:, 1].min()-0.5, X_s[:, 1].max()+0.5
    xx0, xx1 = np.meshgrid(np.linspace(x0_min, x0_max, 200),
                           np.linspace(x1_min, x1_max, 200))
    grid = torch.tensor(np.c_[xx0.ravel(), xx1.ravel()], dtype=torch.float32)
    zz   = log_reg(grid).squeeze().numpy().reshape(xx0.shape)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# 손실 곡선
axes[0].plot(losses, color='steelblue', lw=2)
axes[0].set_title('BCE 손실 수렴')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')

# 결정 경계
axes[1].contourf(xx0, xx1, zz, levels=50, cmap='RdYlBu_r', alpha=0.6)
axes[1].contour(xx0, xx1, zz, levels=[0.5], colors='black', linewidths=2)
for c, color, label in zip([0, 1], colors_cls, labels_cls):
    mask = y_bin == c
    axes[1].scatter(X_s[mask, 0], X_s[mask, 1],
                    color=color, label=label, s=50, edgecolors='k', alpha=0.9, zorder=3)
axes[1].set_title('로지스틱 회귀 결정 경계\n(검은 선: $\\hat{p}=0.5$)')
axes[1].set_xlabel('꽃잎 길이 (표준화)')
axes[1].set_ylabel('꽃잎 너비 (표준화)')
axes[1].legend()
plt.tight_layout()
plt.show()

# 정확도
with torch.no_grad():
    pred_te = (log_reg(X_te_t).squeeze() >= 0.5).float()
acc = (pred_te == y_te_t).float().mean().item()
print(f'Test Accuracy: {acc:.4f} ({acc*100:.1f}%)')

---
## Part 2. 다중 클래스 분류 — 소프트맥스와 크로스 엔트로피

### 2-1. 소프트맥스 (Softmax)

클래스가 3개 이상이면 **소프트맥스**로 확률 분포를 만듭니다:

$$\text{softmax}(z_k) = \frac{e^{z_k}}{\sum_{j=1}^{K} e^{z_j}}, \quad k=1,\ldots,K$$

특성:
- 모든 출력 합 = 1 (확률 분포)
- 가장 큰 로짓이 가장 높은 확률
- $K=2$이면 시그모이드와 동일

### 2-2. 크로스 엔트로피 손실 (Cross-Entropy Loss)

$$\mathcal{L}_{\text{CE}} = -\frac{1}{n}\sum_{i=1}^{n} \log \hat{p}_{i, y_i}$$

정답 클래스의 예측 확률이 높을수록 손실이 작아집니다.

In [ ]:
# 소프트맥스 동작 확인
logits = torch.tensor([[2.0, 1.0, 0.1],    # 클래스 0이 높음
                        [0.5, 2.5, 0.3],    # 클래스 1이 높음
                        [0.1, 0.2, 3.0]])   # 클래스 2가 높음

probs = F.softmax(logits, dim=1)
print('로짓 → 소프트맥스 확률:')
for i, (l, p) in enumerate(zip(logits, probs)):
    print(f'  샘플{i}: logits={l.tolist()}  →  probs={p.detach().numpy().round(3).tolist()}')
    print(f'         예측 클래스={p.argmax().item()},  합={p.sum().item():.4f}')

# 크로스 엔트로피 손실
targets = torch.tensor([0, 1, 2])   # 각 샘플의 정답
ce_loss = nn.CrossEntropyLoss()(logits, targets)
print(f'\nCross-Entropy Loss: {ce_loss.item():.4f}')

# 시각화: 소프트맥스 확률 분포
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
class_names = ['Setosa', 'Versicolor', 'Virginica']
for i, (l, p) in enumerate(zip(logits, probs)):
    colors_bar = ['steelblue', 'tomato', 'seagreen']
    bars = axes[i].bar(class_names, p.detach().numpy(),
                       color=colors_bar, edgecolor='k', alpha=0.8)
    axes[i].set_ylim(0, 1.1)
    axes[i].set_title(f'샘플 {i} (정답: {class_names[targets[i]]})')
    axes[i].set_ylabel('확률')
    # 정답 막대 강조
    bars[targets[i]].set_edgecolor('gold')
    bars[targets[i]].set_linewidth(3)
    for bar, val in zip(bars, p.detach().numpy()):
        axes[i].text(bar.get_x() + bar.get_width()/2, val + 0.02,
                     f'{val:.2f}', ha='center', fontsize=9)
plt.suptitle('소프트맥스 확률 분포 (금색 테두리 = 정답 클래스)', y=1.04)
plt.tight_layout()
plt.show()

---
## Part 3. PyTorch로 Iris 3-클래스 분류기 구현

### 3-1. 데이터 준비

Iris 전체 특성(4개)으로 3종류 붓꽃을 분류합니다:
- **Setosa (0)**: 꽃잎 작고 선명히 구분
- **Versicolor (1)**: 중간
- **Virginica (2)**: 꽃잎 크고 Versicolor와 겹침

In [ ]:
# 전체 Iris 데이터 (4특성, 3클래스)
X_full = iris.data.astype(np.float32)
y_full = iris.target.astype(np.int64)

X_tr3, X_te3, y_tr3, y_te3 = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
)

sc3 = StandardScaler()
X_tr3_s = sc3.fit_transform(X_tr3).astype(np.float32)
X_te3_s = sc3.transform(X_te3).astype(np.float32)

# 텐서 변환
X_tr3_t = torch.tensor(X_tr3_s)
y_tr3_t = torch.tensor(y_tr3)
X_te3_t = torch.tensor(X_te3_s)
y_te3_t = torch.tensor(y_te3)

print(f'Train: {len(X_tr3_s)}개,  Test: {len(X_te3_s)}개')
print(f'클래스: {iris.target_names.tolist()}')

# 특성 쌍 산점도
df_iris = pd.DataFrame(X_full, columns=iris.feature_names)
df_iris['species'] = [iris.target_names[t] for t in y_full]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
palette = {'setosa':'steelblue', 'versicolor':'tomato', 'virginica':'seagreen'}
for ax, (fx, fy) in zip(axes, [('petal length (cm)', 'petal width (cm)'),
                                 ('sepal length (cm)', 'sepal width (cm)')]):
    for sp, color in palette.items():
        m = df_iris['species'] == sp
        ax.scatter(df_iris.loc[m, fx], df_iris.loc[m, fy],
                   color=color, label=sp, s=50, edgecolors='k', alpha=0.8)
    ax.set_xlabel(fx)
    ax.set_ylabel(fy)
    ax.legend(fontsize=8)
plt.suptitle('Iris 3-클래스 분포', y=1.02)
plt.tight_layout()
plt.show()

### 3-2. 소프트맥스 분류기 모델

```
입력 (4) → Linear(4→16) → ReLU → Linear(16→3)
                                    ↓
                              CrossEntropyLoss (내부에 Softmax 포함)
```

> **주의:** `nn.CrossEntropyLoss`는 내부적으로 소프트맥스를 포함하므로  
> 모델 마지막 레이어에 `Softmax`를 **붙이지 않습니다**.

In [ ]:
class SoftmaxClassifier(nn.Module):
    """4 → 16 → 3 분류기"""
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3),    # 소프트맥스 없음 (CrossEntropyLoss에 포함)
        )

    def forward(self, x):
        return self.net(x)       # 로짓(logits) 반환


model3   = SoftmaxClassifier()
crit3    = nn.CrossEntropyLoss()
opt3     = torch.optim.Adam(model3.parameters(), lr=0.02)

train_losses3, train_accs3 = [], []

EPOCHS = 200
for epoch in range(1, EPOCHS + 1):
    model3.train()
    opt3.zero_grad()
    logits3 = model3(X_tr3_t)
    loss3   = crit3(logits3, y_tr3_t)
    loss3.backward()
    opt3.step()

    # 정확도 계산
    with torch.no_grad():
        preds_tr = logits3.argmax(dim=1)
        acc_tr   = (preds_tr == y_tr3_t).float().mean().item()
    train_losses3.append(loss3.item())
    train_accs3.append(acc_tr)

    if epoch % 50 == 0:
        print(f'Epoch {epoch:3d} | Loss: {loss3.item():.4f} | Train Acc: {acc_tr:.4f}')

# Test 정확도
model3.eval()
with torch.no_grad():
    preds_te3 = model3(X_te3_t).argmax(dim=1)
test_acc3 = (preds_te3 == y_te3_t).float().mean().item()
print(f'\nTest Accuracy: {test_acc3:.4f} ({test_acc3*100:.1f}%)')

In [ ]:
# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses3, color='steelblue', lw=2)
axes[0].set_title('학습 손실 (Cross-Entropy)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_accs3, color='seagreen', lw=2)
axes[1].axhline(test_acc3, color='tomato', linestyle='--', lw=2,
                label=f'Test Acc = {test_acc3:.2f}')
axes[1].set_title('학습 정확도')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_ylim(0, 1.05)
axes[1].legend()

plt.tight_layout()
plt.show()

---
## Part 4. 평가 지표 (Evaluation Metrics)

### 4-1. 혼동 행렬 (Confusion Matrix)

분류 결과를 정답 vs 예측 행렬로 표현합니다.

| | 예측 양성(1) | 예측 음성(0) |
|---|---|---|
| **실제 양성(1)** | TP (진양성) | FN (위음성) |
| **실제 음성(0)** | FP (위양성) | TN (진음성) |

### 4-2. 주요 지표

| 지표 | 수식 | 의미 |
|---|---|---|
| **Accuracy** | $(TP+TN)/(P+N)$ | 전체 중 올바른 예측 비율 |
| **Precision** | $TP/(TP+FP)$ | 양성 예측 중 진짜 양성 비율 |
| **Recall** | $TP/(TP+FN)$ | 실제 양성 중 올바르게 예측한 비율 |
| **F1** | $2 \cdot P \cdot R / (P+R)$ | Precision과 Recall의 조화 평균 |

In [ ]:
# Iris 3-클래스 평가 지표
y_pred_np = preds_te3.numpy()
y_true_np = y_te3_t.numpy()

# 혼동 행렬
cm = confusion_matrix(y_true_np, y_pred_np)

# 지표 출력
print('[ 분류 평가 지표 ]')
print(f'  Accuracy : {accuracy_score(y_true_np, y_pred_np):.4f}')
print(f'  Precision (macro): {precision_score(y_true_np, y_pred_np, average="macro"):.4f}')
print(f'  Recall    (macro): {recall_score(y_true_np, y_pred_np, average="macro"):.4f}')
print(f'  F1        (macro): {f1_score(y_true_np, y_pred_np, average="macro"):.4f}')

print('\n[ 클래스별 지표 ]')
print(f'{"":15} Precision  Recall    F1')
for i, name in enumerate(iris.target_names):
    p = precision_score(y_true_np, y_pred_np, average=None)[i]
    r = recall_score(y_true_np, y_pred_np, average=None)[i]
    f = f1_score(y_true_np, y_pred_np, average=None)[i]
    print(f'  {name:<13} {p:.4f}     {r:.4f}    {f:.4f}')

# 혼동 행렬 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names,
            ax=axes[0], linewidths=0.5, linecolor='gray')
axes[0].set_title('혼동 행렬 (Confusion Matrix)')
axes[0].set_xlabel('예측 클래스')
axes[0].set_ylabel('실제 클래스')

# 정규화 혼동 행렬 (비율)
cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names,
            ax=axes[1], linewidths=0.5, linecolor='gray',
            vmin=0, vmax=1)
axes[1].set_title('정규화 혼동 행렬 (비율)')
axes[1].set_xlabel('예측 클래스')
axes[1].set_ylabel('실제 클래스')

plt.tight_layout()
plt.show()

### 4-3. Precision vs Recall 트레이드오프와 ROC 곡선

임계값(threshold)을 바꾸면 Precision과 Recall이 반대 방향으로 움직입니다.

- **임계값 ↑** → Precision↑, Recall↓ (확신하는 것만 양성 예측)
- **임계값 ↓** → Precision↓, Recall↑ (더 많이 양성 예측)

**ROC-AUC**: 모든 임계값에서의 성능을 요약
- **AUC = 1.0**: 완벽한 분류기
- **AUC = 0.5**: 랜덤 분류기

In [ ]:
# 이진 분류 (Setosa vs Non-Setosa)로 ROC 곡선 분석
log_reg.eval()
with torch.no_grad():
    probs_te = log_reg(X_te_t).squeeze().numpy()   # 클래스 1 확률

fpr, tpr, thresholds = roc_curve(y_te, probs_te)
auc = roc_auc_score(y_te, probs_te)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# ROC 곡선
axes[0].plot(fpr, tpr, color='steelblue', lw=2.5, label=f'ROC (AUC={auc:.4f})')
axes[0].plot([0,1],[0,1], 'k--', lw=1.5, label='랜덤 분류기 (AUC=0.5)')
axes[0].fill_between(fpr, tpr, alpha=0.15, color='steelblue')
axes[0].set_title('ROC 곡선 (Setosa vs Non-Setosa)')
axes[0].set_xlabel('False Positive Rate (FPR)')
axes[0].set_ylabel('True Positive Rate (Recall)')
axes[0].legend()

# 임계값별 Precision-Recall 트레이드오프
thrs = np.linspace(0.05, 0.95, 50)
precs, recs, f1s = [], [], []
for thr in thrs:
    pred_thr = (probs_te >= thr).astype(int)
    if pred_thr.sum() == 0:
        precs.append(0.0)
    else:
        precs.append(precision_score(y_te, pred_thr, zero_division=0))
    recs.append(recall_score(y_te, pred_thr, zero_division=0))
    p, r = precs[-1], recs[-1]
    f1s.append(2*p*r/(p+r) if (p+r) > 0 else 0)

axes[1].plot(thrs, precs, color='steelblue', lw=2, label='Precision')
axes[1].plot(thrs, recs,  color='tomato',    lw=2, label='Recall')
axes[1].plot(thrs, f1s,   color='seagreen',  lw=2, label='F1')
axes[1].axvline(0.5, color='gray', linestyle='--', lw=1.5, label='임계값=0.5')
axes[1].set_title('임계값에 따른 Precision / Recall / F1')
axes[1].set_xlabel('임계값 (Threshold)')
axes[1].set_ylabel('점수')
axes[1].set_ylim(0, 1.05)
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'AUC = {auc:.4f}')
best_thr_idx = np.argmax(f1s)
print(f'최적 임계값(F1 기준): {thrs[best_thr_idx]:.2f}  F1={f1s[best_thr_idx]:.4f}')

---
## Exercise — MNIST 소프트맥스 분류기

> **MNIST**: 0~9 손글씨 숫자 이미지 (28×28 픽셀, 10 클래스)

아래 셀들을 순서대로 완성하여 MNIST 분류기를 구축하세요.

In [ ]:
# MNIST 데이터 로드 (처음 실행 시 다운로드, 약 10~30초)
try:
    from torchvision import datasets, transforms
    transform = transforms.ToTensor()
    mnist_tr = datasets.MNIST(root='/tmp/mnist', train=True,  download=True, transform=transform)
    mnist_te = datasets.MNIST(root='/tmp/mnist', train=False, download=True, transform=transform)

    # 실습을 위해 일부만 사용 (각 클래스 당 600/100 개)
    # 전체를 사용하고 싶으면 아래 슬라이싱을 제거하세요
    X_mn_tr = mnist_tr.data[:6000].float().view(-1, 784) / 255.0
    y_mn_tr = mnist_tr.targets[:6000]
    X_mn_te = mnist_te.data[:1000].float().view(-1, 784) / 255.0
    y_mn_te = mnist_te.targets[:1000]
    print(f'Train: {X_mn_tr.shape},  Test: {X_mn_te.shape}')
    MNIST_LOADED = True
except ImportError:
    # torchvision 없을 경우 sklearn으로 fallback
    print('torchvision 없음 → sklearn MNIST 사용')
    from sklearn.datasets import fetch_openml
    mnist_sk = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
    Xm = mnist_sk.data[:7000].astype(np.float32) / 255.0
    ym = mnist_sk.target[:7000].astype(np.int64)
    X_mn_tr = torch.tensor(Xm[:6000])
    y_mn_tr = torch.tensor(ym[:6000])
    X_mn_te = torch.tensor(Xm[6000:])
    y_mn_te = torch.tensor(ym[6000:])
    MNIST_LOADED = True

# 샘플 이미지 시각화
fig, axes = plt.subplots(2, 10, figsize=(14, 3))
for digit in range(10):
    idx = (y_mn_tr == digit).nonzero(as_tuple=True)[0][0]
    img = X_mn_tr[idx].view(28, 28).numpy()
    axes[0, digit].imshow(img, cmap='gray')
    axes[0, digit].set_title(str(digit))
    axes[0, digit].axis('off')
    axes[1, digit].hist(img.ravel(), bins=20, color='steelblue')
    axes[1, digit].axis('off')
plt.suptitle('MNIST 각 클래스 샘플 이미지 (위: 이미지, 아래: 픽셀 분포)', y=1.04)
plt.tight_layout()
plt.show()

### Exercise 1. MNIST 소프트맥스 분류기 완성

아래 모델과 학습 루프를 완성하세요.

**요구 사항:**
- 모델 구조: `784 → 128 → ReLU → 64 → ReLU → 10`
- 손실 함수: `CrossEntropyLoss`
- 옵티마이저: `Adam`, `lr=0.001`
- DataLoader 배치 크기: `256`
- 학습: `20 에폭`
- 매 5 에폭마다 Train Loss와 Test Accuracy 출력

In [ ]:
# Exercise 1: MNIST 분류기 완성
# Your code here

class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # Your code here: 784 → 128 → ReLU → 64 → ReLU → 10
        )

    def forward(self, x):
        # Your code here
        pass


# 데이터로더, 손실, 옵티마이저, 학습 루프
# Your code here

### Exercise 2. 혼동 행렬과 오분류 샘플 분석

학습한 MNIST 모델로:
1. Test 셋 혼동 행렬을 시각화하세요.
2. **가장 많이 혼동된 두 클래스** (예: 4↔9, 3↔8)를 찾아 오분류된 이미지 5장을 보여주세요.

In [ ]:
# Exercise 2: 혼동 행렬 + 오분류 샘플
# Your code here

### Exercise 3. (도전) 결정 경계 시각화

Iris 데이터에서 **꽃잎 길이, 꽃잎 너비** 2개 특성만 사용해  
3-클래스 소프트맥스 분류기를 학습하고 결정 경계를 시각화하세요.

힌트: `plt.contourf`로 그리드 전체에 대한 예측 클래스를 색으로 표시

In [ ]:
# Exercise 3: 3-클래스 결정 경계 시각화
# Your code here

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **시그모이드** | $\sigma(z) = 1/(1+e^{-z})$, 이진 분류 확률 출력 |
| **결정 경계** | $\mathbf{w}^\top\mathbf{x} + b = 0$ — 클래스 구분선 |
| **BCE Loss** | $-[y\log\hat{p} + (1-y)\log(1-\hat{p})]$ |
| **소프트맥스** | $e^{z_k} / \sum e^{z_j}$ — 다중 클래스 확률 |
| **CrossEntropyLoss** | `nn.CrossEntropyLoss` = Softmax + NLLLoss (내부 포함) |
| **Accuracy** | 전체 예측 중 올바른 비율 |
| **Precision** | 양성 예측 중 진짜 양성 비율 (FP 최소화) |
| **Recall** | 실제 양성 중 올바르게 찾은 비율 (FN 최소화) |
| **F1** | Precision·Recall 조화 평균 |
| **ROC-AUC** | 임계값 무관 종합 성능 지표 (1.0이 최고) |
| **혼동 행렬** | 클래스별 예측 오류 분포 파악 |

---

**다음 강의 (Week 7):** 신경망 — 다층 퍼셉트론, 활성화 함수, 역전파